### Brief inplementatin of the LangGraph

Our solution incorporates a complex LangGraph orchestrated flow, keeping the User interactively in the Loop.

The user provides input, which is then processed by the **Retriever** and checked against a **Prompt Verificator** to determine whether the context is sufficient and relevant. If not, the user is guided to cooperate until a good similarity threshold is met.

If the context is sufficient, the **Writer** drafts a story and sends it to the **Watcher**, who is responsible for monitoring the process, keeping memory of iterations, and interacting with the user.

The Draft is sent to the **Editor**, who checks against storytelling protocols and searches for grammar/lexical mistakes. Multiple loops are executed until the protocols are met.

During contextual and character embedding, different LLM models are used depending on their limit and inference speed. Our Final Editor utilizes a fine-tuned Ollama-serviced BGgpt model aligned with best editor practices.

1. **User Interaction**:
   - The user provides input, such as a request for a story about a specific character or theme.
   - The system may ask for additional personalized details like the main character, environment, or moral/lesson.

2. **Retriever**:
   - The input is processed by the Retriever, which uses a Vector DB with contextual and character LLM-enhanced embeddings.
   - It checks if the context is enough and relevant.

3. **Prompt Verificator**:
   - If the context is not sufficient, the Prompt Verificator checks the similarity threshold and may prompt the user for more information.

4. **Writer**:
   - If the context is sufficient, the Writer drafts a story based on the input.

5. **Watcher**:
   - The Watcher monitors the process, updating the state and allowing for a maximum of three drafts.
   - It interacts with the user if necessary.

6. **Editor**:
   - The draft is checked by the Editor against storytelling protocols and for grammar/lexical mistakes using a BGgpt model serviced by Ollama.

7. **Success**:
   - If the draft passes the checks, the process is marked as successful.

This flowchart outlines a structured approach to generating personalized stories, ensuring quality and relevance through multiple stages of verification and editing.

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain-groq==0.2.0 --progress-bar off
!pip install -qqq langgraph==0.2.22 --progress-bar off

In [ ]:
import sqlite3
import textwrap
from enum import Enum, auto
from typing import List, Literal, Optional, TypedDict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab import userdata
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph import END, StateGraph
from pydantic import BaseModel, Field

np.random.seed(42)

MODEL = "llama-3.1-70b-versatile"

llm = ChatGroq(temperature=0, model_name=MODEL, api_key=userdata.get("GROQ_API_KEY"))

EDITOR_PROMPT = """
Rewrite for maximum social media engagement:

- Use attention-grabbing, concise language
- Inject personality and humor
- Optimize formatting (short paragraphs)
- Encourage interaction (questions, calls-to-action)
- Ensure perfect grammar and spelling
- Rewrite from first person perspective, when talking to an audience

Use only the information provided in the text. Think carefully.
"""

TWITTER_PROMPT = """
Generate a high-engagement tweet from the given text:
1. What problem does this solve?
2. Focus on the main technical points/features
3. Write a short, coherent paragraph (2-3 sentences max)
4. Use natural, conversational language
5. Optimize for virality: make it intriguing, relatable, or controversial
6. Exclude emojis and hashtags
"""

TWITTER_CRITIQUE_PROMPT = """
You are a Tweet Critique Agent. Your task is to analyze tweets and provide actionable feedback to make them more engaging. Focus on:

1. Clarity: Is the message clear and easy to understand?
2. Hook: Does it grab attention in the first few words?
3. Brevity: Is it concise while maintaining impact?
4. Call-to-action: Does it encourage interaction or sharing?
5. Tone: Is it appropriate for the intended audience?
6. Storytelling: Does it evoke curiosity?
7. Remove hype: Does it promise more than it delivers?

Provide 2-3 specific suggestions to improve the tweet's engagement potential.
Do not suggest hashtags. Keep your feedback concise and actionable.

Your goal is to help the writer improve their social media writing skills and increase engagement with their posts.
"""

# Graph

class Post(BaseModel):
    """A post written in different versions"""

    drafts: List[str]
    feedback: Optional[str]


class AppState(TypedDict):
    user_text: str
    target_audience: str
    edit_text: str
    tweet: Post
    linkedin_post: Post
    n_drafts: int

# Nodes

# Editor
def editor_node(state: AppState):
    prompt = f"""
text:
```
{state["user_text"]}
```
""".strip()
    response = llm.invoke([SystemMessage(EDITOR_PROMPT), HumanMessage(prompt)])
    return {"edit_text": response.content}


# Tweet Writer
def tweet_writer_node(state: AppState):

    post = state["tweet"]

    feedback_prompt = (
        ""
        if not post.feedback
        else f"""
Tweet:
```
{post.drafts[-1]}
```

Use the feedback to improve it:
```
{post.feedback}
```
""".strip()
    )

    prompt = f"""
text:
```
{state["edit_text"]}
```

{feedback_prompt}

Target audience: {state["target_audience"]}

Write only the text for the post
""".strip()

    response = llm.invoke([SystemMessage(TWITTER_PROMPT), HumanMessage(prompt)])
    post.drafts.append(response.content)
    return {"tweet": post}

# Tweet Critique
def critique_tweet_node(state: AppState):
    post = state["tweet"]

    prompt = f"""
Full post:
```
{state["edit_text"]}
```

Suggested tweet (critique this):
```
{post.drafts[-1]}
```

Target audience: {state["target_audience"]}
""".strip()

    response = llm.invoke(
        [SystemMessage(TWITTER_CRITIQUE_PROMPT), HumanMessage(prompt)]
    )
    post.feedback = response.content
    return {"tweet": post}


def supervisor_node(state: AppState):
    return state

# Edges
def should_rewrite(
    state: AppState,
) -> Literal[["tweet_critique"], END]:
    tweet = state["tweet"]
    if len(tweet.drafts) >= n_drafts:
        return END

    return ["tweet_critique"]

graph = StateGraph(AppState)

graph.add_node("editor", editor_node)
graph.add_node("tweet_writer", tweet_writer_node)
graph.add_node("tweet_critique", critique_tweet_node)
graph.add_node("supervisor", supervisor_node)

graph.add_edge("editor", "tweet_writer")

graph.add_edge("tweet_writer", "supervisor")
graph.add_conditional_edges("supervisor", should_rewrite)

graph.add_edge("tweet_critique", "tweet_writer")

graph.set_entry_point("editor")

app = graph.compile()

display(Image(app.get_graph().draw_mermaid_png()))

%%time
config = {"configurable": {"thread_id": 42}}

user_text = """
With 22 billion parameters, Mistral Small v24.09 offers customers a convenient mid-point between Mistral NeMo 12B and Mistral Large 2,
providing a cost-effective solution that can be deployed across various platforms and environments.
The new small model delivers significant improvements in human alignment, reasoning capabilities, and code over the previous model.

Mistral-Small-Instruct-2409 is an instruct fine-tuned version with the following characteristics:

- 22B parameters
- Vocabulary to 32768
- Supports function calling
- 128k sequence length

Mistral Small v24.09 is released under the MRL license. You may self-deploy it for non-commercial purposes, using e.g. vLLM

Weights on HuggingFace hub: https://huggingface.co/mistralai/Mistral-Small-Instruct-2409
"""

state = app.invoke(
    {
        "user_text": user_text,
        "target_audience": "AI/ML engineers and researchers, Data Scientists",
        "tweet": Post(drafts=[], feedback=None),
        "linkedin_post": Post(drafts=[], feedback=None),
        "n_drafts": 3,
    },
    config=config,
)

print(state["tweet"].feedback)

In [ ]:
print(state["edit_text"])

for i, draft in enumerate(state["tweet"].drafts):
    print(f"Draft #{i+1}")
    print("-" * 10)
    print(textwrap.fill(draft, 80))
    print()
print(state["edit_text"])
for i, draft in enumerate(state["tweet"].drafts):
    print(f"Draft #{i+1}")
    print("-" * 10)
    print(textwrap.fill(draft, 80))
    print()

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [1]:
import textwrap
from typing import List, Literal, Optional, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI
from langgraph.graph import END, StateGraph
from pydantic import BaseModel
import json
import os

# Load configuration
with open('../config.json') as config_file:
    config = json.load(config_file)
    os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set up ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    api_key=OPENAI_API_KEY,
    temperature=0.7,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Prompts
WRITER_PROMPT = """
Generate a fairy tale based on the given input and personalized details:
- Incorporate the main character, environment, and moral lesson
- Use vivid, engaging language suitable for the target audience
- Ensure the story has a clear beginning, middle, and end
- Include elements of wonder and magic typical in fairy tales
- Aim for a length of about 300 words
"""

EDITOR_PROMPT = """
Review and improve the fairy tale:
- Check for consistency with the personalized details
- Ensure the story follows the fairy tale structure
- Improve language and pacing
- Enhance character development and world-building
- Strengthen the moral lesson
"""

WATCHER_PROMPT = """
Analyze the fairy tale and provide feedback:
1. Engagement: Is the story captivating from the beginning?
2. Character development: Are the characters well-defined and relatable?
3. Plot: Is the story coherent and well-paced?
4. Setting: Is the fairy tale world vividly described?
5. Moral lesson: Is the intended lesson clear and impactful?
6. Language: Is the writing style appropriate for the target audience?

Provide 2-3 specific suggestions to improve the story.
Keep your feedback concise and actionable.
"""

# Define state and models
class FairyTale(BaseModel):
    """A fairy tale written in different versions"""
    drafts: List[str]
    feedback: Optional[str]

class AppState(TypedDict):
    user_input: str
    personalized_details: dict
    fairy_tale: FairyTale
    n_drafts: int
    max_drafts: int

# Node functions
def writer_node(state: AppState):
    print(f"\n--- Writer Node (Draft #{state['n_drafts'] + 1}) ---")
    post = state["fairy_tale"]
    
    feedback_prompt = (
        ""
        if not post.feedback
        else f"Previous feedback:\n{post.feedback}\n\nUse this feedback to improve the story."
    )

    prompt = f"""
User input: {state["user_input"]}
Personalized details: {state["personalized_details"]}

{feedback_prompt}

Write a fairy tale based on this information.
"""
    response = llm.invoke([SystemMessage(WRITER_PROMPT), HumanMessage(prompt)])
    post.drafts.append(response.content)
    print("New draft created:")
    print(textwrap.fill(response.content, width=80))
    return {"fairy_tale": post}

def editor_node(state: AppState):
    print(f"\n--- Editor Node (Draft #{state['n_drafts'] + 1}) ---")
    post = state["fairy_tale"]
    prompt = f"""
Fairy tale draft:
{post.drafts[-1]}

Personalized details: {state["personalized_details"]}

Review and improve the fairy tale.
"""
    response = llm.invoke([SystemMessage(EDITOR_PROMPT), HumanMessage(prompt)])
    post.drafts[-1] = response.content
    print("Edited draft:")
    print(textwrap.fill(response.content, width=80))
    return {"fairy_tale": post}

def watcher_node(state: AppState):
    print(f"\n--- Watcher Node (Draft #{state['n_drafts'] + 1}) ---")
    post = state["fairy_tale"]
    prompt = f"""
Fairy tale:
{post.drafts[-1]}

Personalized details: {state["personalized_details"]}

Analyze and provide feedback on this fairy tale.
"""
    response = llm.invoke([SystemMessage(WATCHER_PROMPT), HumanMessage(prompt)])
    post.feedback = response.content
    print("Feedback:")
    print(textwrap.fill(response.content, width=80))
    return {"fairy_tale": post}

# Edges
def should_rewrite(state: AppState) -> Literal["writer", END]:
    state["n_drafts"] += 1
    print(f"\nDraft #{state['n_drafts']} completed.")
    if state["n_drafts"] >= state["max_drafts"]:
        print("Maximum number of drafts reached. Ending process.")
        return END
    print(f"Continuing to Draft #{state['n_drafts'] + 1}.")
    return "writer"

# Build the graph
graph = StateGraph(AppState)

graph.add_node("writer", writer_node)
graph.add_node("editor", editor_node)
graph.add_node("watcher", watcher_node)

graph.set_entry_point("writer")
graph.add_edge("writer", "editor")
graph.add_edge("editor", "watcher")
graph.add_conditional_edges("watcher", should_rewrite)

app = graph.compile()

# Run the fairy tale generator
config = {"configurable": {"thread_id": 42}}

user_input = """I want a story for the rabbit Mitko and the cat Yasen. Setting: Magical forest. Moral lesson: The Good always wins."""

personalized_details = {
    "main_characters": ["Mitko the rabbit", "Yasen the cat"],
    "setting": "Magical forest",
    "moral_lesson": "The Good always wins"
}

initial_state = {
    "user_input": user_input,
    "personalized_details": personalized_details,
    "fairy_tale": FairyTale(drafts=[], feedback=None),
    "n_drafts": 0,
    "max_drafts": 3
}

print("Starting fairy tale generation process...")
state = app.invoke(initial_state, config=config)

print("\nFinal Fairy Tale:")
print(textwrap.fill(state["fairy_tale"].drafts[-1], width=80))

print(f"\nTotal number of drafts: {state['n_drafts']}")

print("\nGeneration process complete.")

C:\Users\a1bg532573\AppData\Local\Temp\ipykernel_21020\935491472.py:18: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


Starting fairy tale generation process...

--- Writer Node (Draft #1) ---


Our solution incorporates a complex LangGraph orchestrated flow, keeping the User interactively in the Loop.

The user provides input, which is then processed by the **Retriever** and checked against a **Prompt Verificator** to determine whether the context is sufficient and relevant. If not, the user is guided to cooperate until a good similarity threshold is met.

If the context is sufficient, the **Writer** drafts a story and sends it to the **Watcher**, who is responsible for monitoring the process, keeping memory of iterations, and interacting with the user.

The Draft is sent to the **Editor**, who checks against storytelling protocols and searches for grammar/lexical mistakes. Multiple loops are executed until the protocols are met.

During contextual and character embedding, different LLM models are used depending on their limit and inference speed. Our Final Editor utilizes a fine-tuned Ollama-serviced BGgpt model aligned with best editor practices.